In [9]:
!pip install pyspark

import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# 1. Leia o arquivo 'videos-preparados.snappy.parquet' no dataframe 'df_video':

spark = SparkSession.builder.appName("Preparação de Dados").getOrCreate()
caminho_arquivo = "/content/videos-preparados.snappy.parquet"

df_video = spark.read.parquet(caminho_arquivo)

df_video.show(5)

# 2. Calcule a quantidade de registros para cada valor único da coluna "Keyword":

keyword_counts = df_video.groupBy("Keyword").count()
keyword_counts.show(5)

# 3. Calcule a média da coluna "Interaction" para cada valor único da coluna 'Keyword':

media_keyword = df_video.groupBy("Keyword").agg(avg("Interaction").alias("Avg_Interaction"))
media_keyword.show(5)

# 4. Calcule o valor máximo da coluna "Interaction" para cada valor único da coluna 'Keyword' e
# nomeie de 'Rank Interactions', em seguida ordene pela nova coluna em ordem decrescente:

Rank_Interactions = df_video.groupBy("Keyword").agg(max("Interaction").alias("Rank Interactions"))
Rank_Interactions = Rank_Interactions.orderBy(desc("Rank Interactions"))

Rank_Interactions.show(5)

# 5. Calcule a média e a variância da coluna 'Views' para cada valor único da coluna 'Keyword':

media_variancia_views = df_video.groupBy("Keyword").agg(avg("Views").alias("Avg_Views"), variance("Views").alias("Var_Views"))
media_variancia_views.show(5)

# 6. Calcule a média, o valor mínimo e o valor máximo de 'Views' para cada valor único da coluna 'Keyword', sem casas decimais:

stats_views = df_video.groupBy("Keyword").agg(
    round(avg("Views"), 0).alias("Avg_Views"),
    min("Views").alias("Min_Views"),
    max("Views").alias("Max_Views")
)
stats_views.show(5)

# 7. Mostre o primeiro e o último 'Published At' para cada valor único da coluna 'Keyword':

df_video = df_video.withColumn("Published At", to_date(col("Published At"), "yyyy-MM-dd"))

primeiro_ultimo_publishedat = df_video.groupBy("Keyword").agg(min("Published At").alias("First Published At"), max("Published At").alias("Last Published At"))
primeiro_ultimo_publishedat.show(5)

# 8. Conte todos os 'title' de forma normal e todos os únicos e verifique se há diferença: Sim, há diferença: 1869 -1854 = 15

df_video.select(count("title").alias("Total Titles")).show()
df_video.select(countDistinct("title").alias("Unique Titles")).show()

total_titles = df_video.select(count("title")).collect()[0][0]
unique_titles = df_video.select(countDistinct("title")).collect()[0][0]

print(f"Total de títulos: {total_titles}, Títulos únicos: {unique_titles}, Diferença: {total_titles - unique_titles}")


# 9. Mostre a quantidade de registros ordenados por ano em ordem ascendente:

df_video = df_video.withColumn("Published At", to_date(col("Published At"), "yyyy-MM-dd"))
df_video = df_video.withColumn("Year", year(col("Published At")))

df_video.groupBy("Year").count().orderBy("Year").show()

# 10. Mostre a quantidade de registros ordenados por ano e mês em ordem ascendente:

df_video = df_video.withColumn("Month", month(col("Published At")))

df_video.groupBy("Year", "Month").count().orderBy("Year", "Month").show()

# 11. Calcule a média acumulativa de ‘Likes’ para cada ‘Keyword’ ao longo dos anos:

window_spec = Window.partitionBy("Keyword").orderBy("Year").rowsBetween(Window.unboundedPreceding, Window.currentRow)

media_acumulativa_likes = df_video.withColumn("Cumulative_Avg_Likes", avg("Likes").over(window_spec))
media_acumulativa_likes.select("Keyword", "Year", "Cumulative_Avg_Likes").show()


+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|Month|Keyword Index|        Features PCA|     Features Normal|            Features|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+-----+-------------+--------------------+--------------------+--------------------+
|ASMR MUKBANG DOUB...|--ZI0dSbbNU|  2020-04-18|mukbang|378858|   18860|17975269|   18372987|2020|    4|         30.0|[0.6985786560867407]|[0.02303716158264...|[378858.0,1.79752...|
|Deadly car bomb d...|--hxd1CrOqg|  2022-08-22|   news|  6379|    4853|  808787|     820019|2022|    8|         37.0|[0.8936407990235931]|[3.87946679100418...|[6379.0,808787.0,...|
|How Biden&#39;s s...|--ixiTypG8g|  2022-08-24|   news|  1029|    2347|   97434|     100810|202